In [1]:
from tvb.simulator.simulator import Simulator

import tvb.simulator.models
import tvb.simulator.integrators
import tvb.simulator.coupling
import tvb.simulator.monitors

from tvb.simulator.models.oscillator import Generic2dOscillator
from tvb.simulator.integrators import HeunStochastic
from tvb.simulator.coupling import Linear
from tvb.simulator.monitors import Raw, TemporalAverage
from tvb.datatypes.connectivity import Connectivity

import matplotlib.pyplot as plt
import numpy as np
import math
import pickle

/Users/erichter/Documents/Uni/Charite/tvb/tvb-root/scientific_library/tvb/datatypes/surfaces.py:64: UserWarning: Geodesic distance module is unavailable; some functionality for surfaces will be unavailable.
  warnings.warn(msg)


In [2]:
simulator_setup = {
    "connectivity": "Own",
    "model": "Generic2dOscillator",
    "integrator": "HeunStochastic",
    "coupling": "Difference",
    "monitor": "Raw",
}

In [3]:
# Parameters
simulator_params = {
    "a_sim": 2.0,
    "b_sim": -10.0,
    "c_sim": 0.0,
    "d_sim": 0.02,
    "I_sim": 0.0,
    "nsig": 0.003,
    "dt": 1.0,
    "simulation_length": 1000
}

In [4]:
connectivity = Connectivity()
connectivity.weights = np.array([[0., 2.], [2., 0.]])
connectivity.region_labels = np.array(["R1", "R2"])
connectivity.centres = np.array([[0.1, 0.1, 0.1], [0.2, 0.1, 0.1]])
connectivity.tract_lengths = np.array([[0., 2.5], [2.5, 0.]])
connectivity.configure()

In [5]:
tvb_data = '/Users/erichter/Documents/Uni/Charite/TVB_Distribution/tvb_data/tvb_data/'
conn = Connectivity.from_file(tvb_data + 'connectivity/connectivity_76.zip')
conn.configure()

2022-09-20 16:09:09,083 - WARNING - tvb.basic.readers - File 'hemispheres' not found in ZIP.


In [6]:
oscillator_model = getattr(tvb.simulator.models, simulator_setup["model"])(
    a=np.asarray([simulator_params["a_sim"]]),
    b=np.asarray([simulator_params["b_sim"]]),
    c=np.asarray([simulator_params["c_sim"]]),
    d=np.asarray([simulator_params["d_sim"]]),
    I=np.asarray([simulator_params["I_sim"]]),
)
oscillator_model.configure()

In [7]:
integrator = getattr(tvb.simulator.integrators, simulator_setup["integrator"])(dt=simulator_params["dt"])
integrator.noise.nsig = np.array([simulator_params["nsig"]])
integrator.configure()

In [8]:
coupling = getattr(tvb.simulator.coupling, simulator_setup["coupling"])()

In [9]:
monitor = getattr(tvb.simulator.monitors, simulator_setup["monitor"])()

In [10]:
sim = Simulator(
    model=oscillator_model,
    connectivity=conn,
    coupling=coupling,
    integrator=integrator,
    monitors=(monitor,),
    simulation_length=simulator_params["simulation_length"]
)

In [11]:
sim.configure()

,value
Type,Simulator
conduction_speed,3.0
connectivity,Connectivity gid: 31ddd00b-03c7-4483-a77b-669e24f7e806
coupling,Difference gid: eef05457-89d3-4c1f-97a9-187aeafc54be
gid,UUID('0abbb1ff-0e30-4d66-bedc-599e24e93d32')
initial_conditions,None
integrator,HeunStochastic gid: 5e597595-c80e-4d0e-8dbf-0208ee37aa17
model,Generic2dOscillator gid: 03a5f890-15c0-46bb-b024-29caeaca183f
monitors,"(,)"
simulation_length,1000.0


In [13]:
sim.connectivity.weights.shape

(76, 76)

In [18]:
sim.history.nnz_row_el_idx

array([ 0,  0,  0, ..., 74, 74, 74], dtype=int32)

In [107]:
sim.history.const_indices

array([[[ 0],
        [ 1],
        [ 3],
        ...,
        [72],
        [73],
        [74]]], dtype=int32)

In [110]:
sim.connectivity.idelays

array([[ 0,  7, 11, ..., 30, 33,  0],
       [ 7,  0, 16, ..., 32, 36,  0],
       [11, 16,  0, ..., 30, 30,  0],
       ...,
       [30, 32, 30, ...,  0,  9,  0],
       [33, 36, 30, ...,  9,  0,  0],
       [ 0,  0,  0, ...,  0,  0,  0]])

In [101]:
sim.connectivity.weights[0]

array([2., 2., 0., 2., 0., 0., 2., 0., 0., 0., 2., 2., 0., 0., 0., 0., 0.,
       0., 2., 0., 2., 2., 0., 0., 2., 0., 0., 0., 0., 0., 2., 0., 0., 3.,
       2., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0.])

In [112]:
sim.connectivity.idelays[(sim.connectivity.weights != 0.0)]

array([ 0,  7, 15, ...,  9,  9,  0])

In [98]:
sim.connectivity.weights

array([[2., 2., 0., ..., 0., 0., 0.],
       [3., 2., 0., ..., 0., 0., 0.],
       [0., 0., 2., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 2., 2., 0.],
       [0., 0., 0., ..., 3., 2., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [24]:
step = 5
time_indices = ((step - 1 - sim.history.nnz_idelays + sim.history.n_time) % sim.history.n_time) 
time_indices = time_indices.reshape((-1, 1)) * sim.history.time_stride

In [78]:
time_indices

array([[ 304],
       [3724],
       [3116],
       ...,
       [3572],
       [3572],
       [ 304]], dtype=int32)

In [79]:
time_indices + sim.history.const_indices

array([[[ 304],
        [3725],
        [3119],
        ...,
        [3644],
        [3645],
        [ 378]]], dtype=int32)

In [70]:
-1*np.unique(sim.history.nnz_idelays)

array([  0,  -2,  -3,  -4,  -5,  -6,  -7,  -8,  -9, -10, -11, -12, -13,
       -14, -15, -16, -17, -18, -19, -20, -21, -22, -23, -24, -25, -26,
       -27, -28, -29, -30, -31, -32, -33, -34, -35, -36, -37, -38, -39,
       -42, -43, -44, -45, -46], dtype=int32)

In [113]:
_, idx = np.unique(sim.history.nnz_idelays, return_index=True)

In [128]:
np.unique(sim.history.nnz_idelays)

array([ 0,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 42, 43, 44, 45, 46], dtype=int32)

In [127]:
np.arange(np.unique(sim.history.nnz_idelays).max())

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45])

In [147]:
sim.connectivity.idelays[0]

array([ 0,  7, 11, 15, 14, 12, 17, 11, 15,  8, 12,  6, 18, 13, 19, 20, 19,
       24, 16, 21, 21, 20, 27, 18, 12, 21, 19, 17, 15,  5, 13, 14, 15, 10,
       13, 25, 20,  0, 25, 31, 21, 17, 16, 16, 18, 31, 29, 21, 28, 28, 25,
       32, 30, 23, 30, 31, 30, 25, 23, 25, 32, 33, 22, 27, 22, 34, 29, 29,
       33, 28, 31, 33, 28, 30, 33,  0])

In [146]:
sim.connectivity.weights[0]

array([2., 2., 0., 2., 0., 0., 2., 0., 0., 0., 2., 2., 0., 0., 0., 0., 0.,
       0., 2., 0., 2., 2., 0., 0., 2., 0., 0., 0., 0., 0., 2., 0., 0., 3.,
       2., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0.])

In [155]:
sim.history.const_indices[0, :, 0]

array([ 0,  1,  3, ..., 72, 73, 74], dtype=int32)

In [134]:
sim.history.nnz_idelays

array([ 0,  7, 15, ...,  9,  9,  0], dtype=int32)

In [152]:
sim.history.nnz_col_el_idx

array([ 0,  1,  3, ..., 72, 73, 74], dtype=int32)

In [141]:
X_delayed = X[-1*np.arange(np.unique(sim.history.nnz_idelays).max()+1)][sim.history.nnz_idelays]

In [189]:
taps = list(-1*np.arange(np.unique(sim.history.nnz_idelays).max()+1)-1)
taps

[-1,
 -2,
 -3,
 -4,
 -5,
 -6,
 -7,
 -8,
 -9,
 -10,
 -11,
 -12,
 -13,
 -14,
 -15,
 -16,
 -17,
 -18,
 -19,
 -20,
 -21,
 -22,
 -23,
 -24,
 -25,
 -26,
 -27,
 -28,
 -29,
 -30,
 -31,
 -32,
 -33,
 -34,
 -35,
 -36,
 -37,
 -38,
 -39,
 -40,
 -41,
 -42,
 -43,
 -44,
 -45,
 -46,
 -47]

In [183]:
X_delayed[np.arange(X_delayed.shape[0]), :, sim.history.nnz_col_el_idx, :].reshape()

array([[[ 0.42411244]],

       [[-0.00418759]],

       [[-0.12726091]],

       ...,

       [[ 0.38930377]],

       [[ 0.4203801 ]],

       [[ 2.02527135]]])

In [190]:
X_delayed.shape[0]

1560

In [191]:
sim.history.n_nnzw

1560

In [57]:
taps = list(sim.history.nnz_idelays)

In [59]:
taps.insert(0, 0)

In [60]:
taps

[0,
 0,
 7,
 15,
 17,
 12,
 6,
 16,
 21,
 20,
 12,
 13,
 10,
 13,
 7,
 0,
 12,
 10,
 18,
 21,
 20,
 29,
 25,
 32,
 22,
 15,
 25,
 10,
 5,
 12,
 0,
 3,
 14,
 13,
 11,
 15,
 13,
 15,
 17,
 0,
 14,
 12,
 17,
 19,
 18,
 17,
 18,
 15,
 27,
 23,
 23,
 16,
 14,
 10,
 12,
 18,
 21,
 18,
 19,
 13,
 10,
 20,
 17,
 23,
 26,
 31,
 2,
 14,
 19,
 14,
 0,
 6,
 21,
 20,
 20,
 13,
 14,
 9,
 13,
 29,
 25,
 28,
 34,
 19,
 14,
 19,
 24,
 22,
 18,
 20,
 3,
 6,
 18,
 17,
 12,
 18,
 29,
 23,
 5,
 17,
 23,
 10,
 12,
 21,
 17,
 0,
 16,
 11,
 15,
 32,
 20,
 21,
 9,
 7,
 16,
 16,
 13,
 23,
 15,
 23,
 25,
 32,
 2,
 11,
 12,
 17,
 20,
 20,
 0,
 16,
 12,
 15,
 23,
 26,
 19,
 20,
 23,
 24,
 27,
 16,
 22,
 16,
 17,
 12,
 21,
 20,
 16,
 28,
 34,
 19,
 27,
 0,
 3,
 10,
 28,
 34,
 20,
 8,
 15,
 28,
 27,
 28,
 13,
 7,
 12,
 16,
 10,
 17,
 24,
 20,
 11,
 16,
 15,
 0,
 7,
 25,
 31,
 31,
 30,
 19,
 17,
 14,
 9,
 17,
 9,
 13,
 25,
 9,
 25,
 10,
 7,
 16,
 22,
 30,
 10,
 18,
 17,
 15,
 11,
 7,
 0,
 18,
 25,
 26,
 25,
 23,
 17,

In [31]:
sim.connectivity.idelays

array([[ 0,  7, 11, ..., 30, 33,  0],
       [ 7,  0, 16, ..., 32, 36,  0],
       [11, 16,  0, ..., 30, 30,  0],
       ...,
       [30, 32, 30, ...,  0,  9,  0],
       [33, 36, 30, ...,  9,  0,  0],
       [ 0,  0,  0, ...,  0,  0,  0]])

In [40]:
X[0][:, sim.history.nnz_col_el_idx]

array([[[ 0.42411244],
        [ 3.29855439],
        [ 1.46256772],
        ...,
        [-1.37734923],
        [ 3.00106144],
        [ 2.02527135]]])

In [45]:
sim.coupling._cached_nzr

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51,
       52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68,
       69, 70, 71, 72, 73, 74], dtype=int32)

In [42]:
np.zeros_like(X[0]).shape

(1, 76, 1)

In [12]:
(t, X), = sim.run()

1 (1, 76, 1) (1, 1560, 1)
2 (1, 76, 1) (1, 1560, 1)
3 (1, 76, 1) (1, 1560, 1)
4 (1, 76, 1) (1, 1560, 1)
5 (1, 76, 1) (1, 1560, 1)
6 (1, 76, 1) (1, 1560, 1)
7 (1, 76, 1) (1, 1560, 1)
8 (1, 76, 1) (1, 1560, 1)
9 (1, 76, 1) (1, 1560, 1)
10 (1, 76, 1) (1, 1560, 1)
11 (1, 76, 1) (1, 1560, 1)
12 (1, 76, 1) (1, 1560, 1)
13 (1, 76, 1) (1, 1560, 1)
14 (1, 76, 1) (1, 1560, 1)
15 (1, 76, 1) (1, 1560, 1)
16 (1, 76, 1) (1, 1560, 1)
17 (1, 76, 1) (1, 1560, 1)
18 (1, 76, 1) (1, 1560, 1)
19 (1, 76, 1) (1, 1560, 1)
20 (1, 76, 1) (1, 1560, 1)
21 (1, 76, 1) (1, 1560, 1)
22 (1, 76, 1) (1, 1560, 1)
23 (1, 76, 1) (1, 1560, 1)
24 (1, 76, 1) (1, 1560, 1)
25 (1, 76, 1) (1, 1560, 1)
26 (1, 76, 1) (1, 1560, 1)
27 (1, 76, 1) (1, 1560, 1)
28 (1, 76, 1) (1, 1560, 1)
29 (1, 76, 1) (1, 1560, 1)
30 (1, 76, 1) (1, 1560, 1)
31 (1, 76, 1) (1, 1560, 1)
32 (1, 76, 1) (1, 1560, 1)
33 (1, 76, 1) (1, 1560, 1)
34 (1, 76, 1) (1, 1560, 1)
35 (1, 76, 1) (1, 1560, 1)
36 (1, 76, 1) (1, 1560, 1)
37 (1, 76, 1) (1, 1560, 1)
38 (1, 76,

In [ ]:
X.shape

In [ ]:
f1 = plt.figure(figsize=(14,8))
plt.plot(t, X[:, 0, 0, 0], label="R1")
plt.plot(t, X[:, 0, 1, 0], label="R2")
plt.xlabel("time (ms)")
plt.ylabel("states")
plt.legend()
plt.show()

In [ ]:
#with open("limit-cycle_simulation.pkl", "wb") as f:
#    pickle.dump({**simulator_setup, **simulator_params, "simulation": X}, f)

In [ ]:
#np.save(f"limit-cycle_simulation", X)